## Neo4j 연결하기

In [1]:
!pip install neo4j

In [5]:
!pip install py2neo

In [9]:
del data

In [8]:
from py2neo import Graph

# 'testdb'라는 DB에 연결
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4j1234"), name="testdb")

# 쿼리 실행
data = graph.run("""
    MATCH (c:고객)
    RETURN c.`고객ID` AS 고객ID, c.`고객이름` AS 이름
""").to_data_frame()

print(data.head())

    고객ID     이름
0  C0116  고객116
1  C0169  고객169
2  C0086   고객86
3  C0104  고객104
4  C0126  고객126


* https://pypi.org/project/neo4j/

In [10]:
from neo4j import GraphDatabase
import pandas as pd

# 연결
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "neo4j1234"))

def get_customers(tx):
    query = """
    MATCH (c:고객)
    RETURN c.`고객ID` AS 고객ID, c.`고객이름` AS 이름
    """
    return list(tx.run(query))

with driver.session(database='testdb') as session:
    result = session.execute_read(get_customers)

# Pandas로 변환
df = pd.DataFrame([dict(record) for record in result])
print(df.head())

    고객ID     이름
0  C0116  고객116
1  C0169  고객169
2  C0086   고객86
3  C0104  고객104
4  C0126  고객126


## NetworkX

In [11]:
from neo4j import GraphDatabase
import pandas as pd
import networkx as nx

# 1. Neo4j 연결 설정
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "neo4j1234"))

# 2. 관계 쿼리
def get_edges(tx):
    query = """
    MATCH (a:고객)-[:동반인]->(b:고객)
    RETURN a.`고객ID` AS source, b.`고객ID` AS target
    """
    return list(tx.run(query))

# 3. 데이터 가져오기
with driver.session(database="testdb") as session:  # DB 이름 명시
    result = session.execute_read(get_edges)

df = pd.DataFrame([dict(record) for record in result])
print(df.head())

  source target
0  C0116  C0142
1  C0169  C0115
2  C0086  C0199
3  C0086  C0072
4  C0086  C0153


In [12]:
# 1. 방향 있는 그래프 생성
G = nx.DiGraph()  # ← 방향이 없으면 nx.Graph()

# 2. 간선 추가
G.add_edges_from(zip(df["source"], df["target"]))

In [13]:
# Degree 연결중심성
degree_centrality = nx.degree_centrality(G)

# Betweenness 매개중심성
betweenness_centrality = nx.betweenness_centrality(G)

# closeness 근접중심성
closeness_centrality = nx.closeness_centrality(G)

# PageRank
pagerank = nx.pagerank(G)

# 결과 보기
print("📌 Degree 중심성 상위 5명:")
print(sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5])

📌 Degree 중심성 상위 5명:
[('C0106', 0.05208333333333333), ('C0133', 0.046875), ('C0153', 0.041666666666666664), ('C0163', 0.041666666666666664), ('C0100', 0.041666666666666664)]


In [14]:
G = nx.DiGraph()
G.add_edges_from(zip(df["source"], df["target"]))

out_degree = dict(G.out_degree())
top_out = sorted(out_degree.items(), key=lambda x: x[1], reverse=True)[:10]

print("동반인 많이 데려온 고객 Top 10:")
for node, deg in top_out:
    print(f" - {node}: {deg}명 데려옴")

동반인 많이 데려온 고객 Top 10:
 - C0114: 6명 데려옴
 - C0035: 6명 데려옴
 - C0064: 5명 데려옴
 - C0198: 5명 데려옴
 - C0104: 4명 데려옴
 - C0163: 4명 데려옴
 - C0154: 4명 데려옴
 - C0149: 4명 데려옴
 - C0133: 4명 데려옴
 - C0027: 4명 데려옴
